In [8]:
import numpy as np
import pandas as pd
from binance.client import Client

In [9]:
symbol = "BTCUSDT"
coin = "BTC"
timeFrame = "12h"
startDate = "2018-07-22"
Binance = {"API":"MtK3YvsbesizJw7x8PzRKbcpnVDWE441aiCOjLn5qPBOX9ZLVBWCITgB8LwjBk7A", 
           "Secret":"YDGdHbK5dPkxI1noxuFLWhkE95N9jTJvwvX6uUBwAh0UhtMEzQXDhGCK2fi1aFDS"}

In [10]:
client = Client(Binance["API"], Binance["Secret"])

In [13]:
bars = client.get_historical_klines(symbol=f'{coin}USDT',interval=timeFrame,start_str=startDate)
test_df = pd.DataFrame(bars[:],columns=["timestamp","open","high","low","close","volume", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"])
test_df["date"]=pd.to_datetime(test_df["timestamp"],unit="ms").astype(str)
test_df = test_df.drop(["timestamp", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"], axis=1)
test_df["open"] = pd.to_numeric(test_df["open"])
test_df["high"] = pd.to_numeric(test_df["high"])
test_df["low"] = pd.to_numeric(test_df["low"])
test_df["close"] = pd.to_numeric(test_df["close"])
test_df["volume"] = pd.to_numeric(test_df["volume"])

In [14]:
test_df

,open,high,low,close,volume,date
0,7399.25,7487.49,7339.00,7445.02,12592.755738,2018-07-22 00:00:00
1,7446.00,7582.40,7335.50,7394.79,17677.462702,2018-07-22 12:00:00
2,7394.78,7790.00,7375.00,7695.00,22373.761982,2018-07-23 00:00:00
3,7695.00,7833.00,7638.80,7721.01,19272.584689,2018-07-23 12:00:00
4,7721.65,8280.00,7696.00,8257.99,29216.912970,2018-07-24 00:00:00
...,...,...,...,...,...,...
2919,23728.55,24276.74,22906.19,23223.30,133477.069010,2022-07-20 12:00:00
2920,23223.30,23442.77,22466.34,22606.04,87061.173050,2022-07-21 00:00:00
2921,22607.92,23308.80,22341.46,23152.19,97756.508860,2022-07-21 12:00:00
2922,23152.19,23634.40,22805.89,23610.58,75700.059060,2022-07-22 00:00:00


In [3]:
grid_number = 50
expected_ROI = 0.01
equal_Diff_or_Ratio = "RATIO"
lowest_price, highest_price = 3000, 20000
trading_fee_rate = 0.0001
trading_day = 1

mygrid = []
dif = highest_price - lowest_price
if (equal_Diff_or_Ratio == "DIFF"):
    for i in range(grid_number + 1):
        mygrid.append(lowest_price + i * dif / grid_number)
elif (equal_Diff_or_Ratio == "RATIO"):
    ratio = (highest_price / lowest_price)**(1/grid_number)
    for i in range(grid_number + 1):
        mygrid.append(lowest_price * ratio**i)

[3000.0, 3116.0142101058277, 3236.514852527149, 3361.6754238977214, 3491.676130209611, 3626.7041462734915, 3766.9538852126416, 3912.6272783786494, 4063.934066091854, 4221.092099625125, 4384.327654865778, 4553.875758107241, 4729.980524439538, 4912.895509225806, 5102.884073170906, 5300.21976150775, 5505.186697847291, 5718.079993259245, 5939.206171172548, 6168.883608707295, 6407.442995073618, 6655.227807697479, 6912.594806758934, 7179.914548854862, 7457.5719205257765, 7745.966692414841, 8045.514094857029, 8356.645415727076, 8679.808621407099, 9015.469001767864, 9364.109840092422, 9726.233108906601, 10102.360192718252, 10493.03263870584, 10898.812936437216, 11320.285327741198, 11758.056647898027, 12212.757199359849, 12685.041659259181, 13175.590022012006, 13685.108578372658, 14214.330932350122, 14764.019057449932, 15334.964393762413, 15927.988987476863, 16543.946674462346, 17183.724309619243, 17848.243043771505, 18538.45964993817, 19255.367900893616, 20000.000000000044]


In [ ]:
def buy(money, storage, price, cost):
    return money - cost, storage 

In [25]:
for i in range(len(test_df) - trading_day):
    money = 1000
    storage = 0
    trading_unit = money / grid_number
    grid_position = -1
    for j in range(grid_number + 1):
        if (test_df["open"][i] > mygrid[j]):
            grid_position = j
    # trade or hodl
    for j in range(trading_day):
        if (grid_position == -1 and test_df["high"] > lowest_price):
            for k in range(grid_number + 1):
                if (test_df["high"] < mygrid[k]):
                    money, storage = buy()
    

    # get trading profit, hodling profit, MDD
    break

7399.25
50
